In [1]:
! pip install -q langchain sentence-transformers faiss-cpu langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.0/413.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [2]:
%pip install --upgrade --quiet  langchain sentence_transformers

In [1]:
import pandas as pd
import random

# Define sample data for fraud and non-fraud financial statements
fraud_statements = [
    # Revenue Manipulation
    "The company reported revenue from contracts that were never finalized.",
    "Fake invoices were generated to create the illusion of increased sales.",
    "The company recorded sales transactions with non-existent customers.",
    "Revenue was artificially inflated through backdated contracts.",
    "Future sales were prematurely recognized to meet quarterly revenue targets.",
    "The company engaged in 'cookie jar accounting' to smooth out revenue fluctuations.",
    "High-risk loans were issued to boost short-term revenue numbers.",
    "The company falsely claimed refunds and rebates as additional revenue.",
    "Revenue was recognized before receiving payment from customers.",
    "Artificial discounts were applied to inflate sales figures before audits.",

    # Expense Misclassification
    "Personal luxury travel expenses were recorded as business-related costs.",
    "Consulting fees paid to shell companies were classified as legitimate business expenses.",
    "Significant operational losses were masked by reclassifying them as 'one-time' expenses.",
    "Severance pay and legal settlements were recorded as asset purchases.",
    "Expenses related to unapproved vendor payments were hidden under general business expenses.",
    "Research grants were misused for executive bonuses but classified as R&D expenses.",
    "Marketing expenditures were misclassified as technology investments to show higher profitability.",
    "Payroll taxes were underreported to minimize expense recognition.",
    "Interest expenses on fraudulent loans were not recorded in financial statements.",
    "Bribes and kickbacks were disguised as legitimate business payments.",

    # Undisclosed Liabilities & Debt Concealment
    "Significant loan obligations were kept off the balance sheet to deceive investors.",
    "The company created fake vendors to route funds and hide liabilities.",
    "Undisclosed off-the-books debt was used to finance risky investments.",
    "Unreported pension obligations created a misleading picture of financial stability.",
    "The company hid legal settlement costs from public disclosures.",
    "Debt covenants were intentionally violated but not disclosed in financial reports.",
    "Deferred liabilities were incorrectly recorded as long-term assets.",
    "Bank loans were fraudulently secured using falsified financial statements.",
    "Repayment obligations were omitted to create an illusion of higher liquidity.",
    "The company engaged in fraudulent refinancing schemes to hide financial distress.",

    # Asset Fraud
    "The company recorded non-existent real estate holdings to inflate net worth.",
    "Used vehicles were valued as brand-new assets to overstate balance sheet figures.",
    "Exaggerated property values were used to secure larger loans.",
    "The company included fake intellectual property rights as part of its assets.",
    "Machinery and equipment that had been sold were still reported in financial records.",
    "Valuation reports for company-owned land were manipulated to attract investors.",
    "Obsolete technology was reported as cutting-edge innovation in financial statements.",
    "Fixed asset depreciation was ignored to falsely enhance the company’s financial position.",
    "High-value artwork was purchased using company funds and misclassified as office supplies.",
    "The company inflated the value of cryptocurrency holdings without proper accounting measures.",

    # Cash Flow & Bank Fraud
    "Large wire transfers were conducted without proper documentation.",
    "Multiple accounts were used to create a fraudulent perception of cash reserves.",
    "Loans from shell companies were falsely reported as legitimate revenue.",
    "Payments were made to nonexistent suppliers and funneled to executives.",
    "The company engaged in money laundering through complex offshore transactions.",
    "Bogus transactions were created to show false growth in cash flow.",
    "Employee reimbursement claims were falsified to siphon company funds.",
    "Bank reconciliations were manipulated to hide embezzlement activities.",
    "Undocumented cash withdrawals were made from corporate accounts.",
    "Funds were transferred to personal accounts under the guise of 'strategic investments.'",

    # Stock Market & Investor Fraud
    "Company insiders manipulated stock prices by releasing false financial reports.",
    "Earnings reports were altered to mislead shareholders before a stock sale.",
    "Executives engaged in insider trading without reporting to regulatory authorities.",
    "Stock options were granted based on fabricated financial growth projections.",
    "Corporate executives falsified investor presentations to attract funding.",
    "Market rumors were spread to artificially inflate share prices before selling holdings.",
    "Mergers and acquisitions were falsely announced to manipulate stock value.",
    "Financial forecasts were exaggerated to secure additional investments.",
    "Investors were misled about the company's profitability through altered SEC filings.",
    "The company engaged in pump-and-dump schemes to benefit top executives.",

    # Regulatory & Tax Fraud
    "The company underreported revenue to avoid paying corporate taxes.",
    "Tax returns were falsified to claim non-existent deductions.",
    "Foreign income was not disclosed to evade international tax regulations.",
    "Inflated expenses were reported to decrease taxable income artificially.",
    "The company used tax shelters to hide taxable profits in offshore accounts.",
    "Unreported overseas operations concealed additional revenue streams.",
    "Falsified employee wages were claimed for fraudulent tax credits.",
    "Transfer pricing was manipulated to shift profits to low-tax jurisdictions.",
    "Government grants were misused for unrelated corporate expenditures.",
    "The company failed to remit employee tax withholdings to tax authorities.",

    # Cybersecurity & Digital Fraud
    "Customer payment information was stored insecurely, leading to data breaches.",
    "Fake online transactions were processed to create artificial sales records.",
    "Ransomware attacks were falsely classified as 'IT maintenance expenses.'",
    "Unauthorized digital wallets were linked to corporate accounts for illicit transfers.",
    "Executives authorized illegal cryptocurrency transactions under false pretenses.",
    "Data manipulation tactics were used to alter financial reports in online banking systems.",
    "Cybersecurity breaches were deliberately hidden from regulators to avoid legal scrutiny.",
    "User accounts were artificially created to inflate digital transaction numbers.",
    "Digital audit logs were altered to cover up unauthorized financial transactions.",
    "Illicit AI-powered trading algorithms were deployed to manipulate market prices.",

    # Loan & Credit Fraud
    "Mortgage applications contained falsified income information to secure loans.",
    "The company issued loans to fake business entities created by insiders.",
    "Loan default risks were hidden from investors to maintain credit ratings.",
    "Fraudulent personal loans were granted using stolen identities.",
    "Multiple fraudulent credit lines were opened to siphon bank funds.",
    "Loan guarantees were secured using forged financial statements.",
    "Debt write-offs were falsely categorized as profitable investments.",
    "Customers were charged hidden fees that were not disclosed upfront.",
    "Bankruptcy proceedings were delayed through manipulated financial reports.",
    "High-risk subprime loans were misclassified as prime loans to deceive investors."
]

non_fraud_statements = [
    # Revenue Recognition
    "The company recognizes revenue only when goods or services have been delivered and accepted by customers.",
    "All sales transactions are properly documented with supporting invoices and contracts.",
    "Revenue figures are recorded based on actual transactions without estimations or assumptions.",
    "The company follows accrual accounting principles, ensuring revenue is recognized when earned, not just when received.",
    "Sales records are consistently reconciled with customer payments to prevent discrepancies.",
    "The company ensures that revenue is not prematurely recorded before meeting performance obligations.",
    "Deferred revenue is properly recorded for contracts with ongoing service obligations.",
    "Discounts, returns, and allowances are accounted for in revenue calculations to ensure accuracy.",
    "The financial statements are regularly audited to confirm revenue recognition policies comply with IFRS and GAAP.",
    "The company ensures compliance with industry regulations when recognizing revenue from long-term contracts.",

    # Expense Management
    "All business expenses are properly documented with invoices and receipts.",
    "Research and development costs are capitalized or expensed in accordance with accounting standards.",
    "Executive bonuses and compensation are fully disclosed in financial reports.",
    "Employee reimbursements are verified against company policies before approval.",
    "Operational expenses are clearly categorized and recorded in the appropriate accounting period.",
    "Marketing expenses are properly classified and align with budgetary allocations.",
    "The company adheres to ethical procurement practices, ensuring no unauthorized payments are made.",
    "There is a strict internal audit process to verify expense classifications.",
    "Payroll expenses are reconciled with employee records to ensure accuracy.",
    "All capital expenditures are recorded as assets and depreciated over their useful life in compliance with accounting guidelines.",

    # Asset Management & Balance Sheet Integrity
    "The company’s physical assets, such as buildings and machinery, are regularly appraised and properly recorded.",
    "Depreciation and amortization schedules are followed based on asset lifespans.",
    "Inventory management practices ensure stock levels are accurately recorded and matched with physical counts.",
    "Asset purchases are approved through standardized procurement processes.",
    "Intangible assets such as patents and trademarks are properly valued and disclosed.",
    "The company regularly reviews asset impairment to ensure accurate valuations in financial statements.",
    "Company-owned vehicles and equipment are recorded in asset registers with clear usage logs.",
    "The organization follows best practices in fixed asset accounting, including timely write-offs for obsolete equipment.",
    "Investments are reported at fair market value and are reconciled periodically with financial institutions.",
    "The company has robust policies to prevent asset misappropriation, ensuring accountability at all levels.",

    # Liabilities & Debt Transparency
    "All corporate loans and financial obligations are disclosed in financial statements.",
    "The company maintains a conservative debt-to-equity ratio to ensure financial stability.",
    "Bank borrowings are reconciled with lender statements to confirm accurate reporting.",
    "The financial statements provide a clear breakdown of short-term and long-term liabilities.",
    "The company ensures compliance with all debt covenants and regulatory obligations.",
    "Outstanding liabilities, including vendor payments and lease obligations, are properly accounted for.",
    "The company's credit rating is regularly reviewed to ensure financial health.",
    "Deferred tax liabilities are calculated based on tax laws and reported transparently.",
    "The organization maintains a detailed record of contingent liabilities for proper disclosure.",
    "Bond issuances and debt financing activities are disclosed in shareholder communications and regulatory filings.",

    # Corporate Governance & Compliance
    "Financial disclosures comply with all relevant regulatory requirements and accounting standards.",
    "The company’s board of directors regularly reviews financial reports for accuracy and compliance.",
    "Independent auditors conduct external audits to verify financial transparency.",
    "Whistleblower policies are in place to report any financial irregularities.",
    "Compliance officers ensure adherence to anti-corruption and anti-money laundering regulations.",
    "The company follows strict ethical guidelines to prevent conflicts of interest in financial decisions.",
    "Investor communications include full transparency on financial performance and business risks.",
    "Management ensures compliance with Sarbanes-Oxley Act regulations for financial reporting integrity.",
    "The company undergoes regular internal and external audits to ensure financial transparency.",
    "The CFO and senior executives certify the accuracy of financial reports before filing with regulatory bodies.",

    # Cash Flow & Liquidity Management
    "Cash flow statements accurately reflect inflows and outflows from operational, investing, and financing activities.",
    "The company maintains sufficient cash reserves to cover short-term obligations.",
    "Bank reconciliations are performed regularly to ensure accurate cash balance reporting.",
    "The organization follows structured cash management policies to optimize liquidity.",
    "Dividends and stock buybacks are executed based on financial health assessments.",
    "Short-term investments are properly disclosed and managed within risk parameters.",
    "Corporate treasury functions adhere to risk management best practices for cash handling.",
    "Petty cash transactions are monitored and recorded to prevent unauthorized withdrawals.",
    "The company ensures that working capital remains sufficient for daily operations.",
    "Financial modeling and projections are used to maintain a stable liquidity position.",

    # Investor & Shareholder Transparency
    "Shareholder reports provide detailed breakdowns of financial performance and company strategy.",
    "The company regularly conducts earnings calls to update investors on financial results.",
    "Annual and quarterly financial reports are made publicly available to all stakeholders.",
    "Investor presentations provide clear explanations of revenue streams and cost structures.",
    "The company discloses executive compensation and stock options in its annual filings.",
    "Proxy statements provide transparency into major shareholder votes and board decisions.",
    "Performance metrics such as return on equity (ROE) and earnings per share (EPS) are accurately reported.",
    "Dividend distributions are aligned with financial performance and approved by the board.",
    "The company ensures transparency in mergers, acquisitions, and restructuring activities.",
    "Stock buyback programs are executed based on strategic financial planning and market conditions.",

    # Cybersecurity & Financial Data Integrity
    "The company implements strong cybersecurity protocols to protect financial data.",
    "Multi-factor authentication is used for financial system access to prevent unauthorized transactions.",
    "Encryption techniques are applied to safeguard sensitive customer and transaction data.",
    "Regular audits are conducted to ensure compliance with financial data security regulations.",
    "Transaction logs are monitored to detect and prevent unauthorized access.",
    "All online transactions are verified using secure payment gateways.",
    "Risk assessments are conducted periodically to strengthen cybersecurity measures.",
    "The company employs fraud detection algorithms to identify suspicious activities.",
    "Data integrity checks are performed to ensure no unauthorized modifications to financial records.",
    "Strict access controls are enforced to prevent unauthorized changes to financial databases."
]


In [2]:
# Generate fraud and non-fraud data
fraud_data = [{"text": statement, "fraud_status": "fraud"} for statement in fraud_statements]
non_fraud_data = [{"text": random.choice(non_fraud_statements), "fraud_status": "non-fraud"} for _ in range(60)]

# Combine data into a single dataset
data = fraud_data + non_fraud_data
random.shuffle(data)  # Shuffle data to mix fraud and non-fraud rows

# Create a DataFrame
df = pd.DataFrame(data)

# Save to a CSV file
df.to_csv("financial_statements_fraud_dataset.csv", index=False)

In [3]:
df.head(10)

,text,fraud_status
0,The company ensures that working capital remai...,non-fraud
1,High-value artwork was purchased using company...,fraud
2,The company used tax shelters to hide taxable ...,fraud
3,"The company ensures transparency in mergers, a...",non-fraud
4,The organization maintains a detailed record o...,non-fraud
5,Bond issuances and debt financing activities a...,non-fraud
6,Depreciation and amortization schedules are fo...,non-fraud
7,Bankruptcy proceedings were delayed through ma...,fraud
8,Compliance officers ensure adherence to anti-c...,non-fraud
9,Fake online transactions were processed to cre...,fraud


In [4]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

In [5]:
# Ensure NLTK resources are downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [7]:
df.shape

(150, 2)

In [8]:

# Function to clean text
def clean_text(text):
    # Remove non-ASCII characters
    text = text.encode('ascii', 'ignore').decode()

    # Remove punctuation and numbers
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)

    # Convert to lowercase
    text = text.lower()

    # Tokenize text
    tokens = word_tokenize(text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Join tokens back into text
    cleaned_text = ' '.join(tokens)

    return cleaned_text

# Clean 'Fillings' column
df['Clean_Text'] = df['text'].apply(clean_text)

# Drop original 'Text' column if no longer needed
df.drop(columns=['text'], inplace=True)

# Save cleaned data back to CSV if desired
df.to_csv('cleaned_financial_statements.csv', index=False)

# Example of how the cleaned data looks like
print(df.head())

  fraud_status                                         Clean_Text
0    non-fraud  company ensures working capital remains suffic...
1        fraud  highvalue artwork purchased using company fund...
2        fraud  company used tax shelters hide taxable profits...
3    non-fraud  company ensures transparency mergers acquisiti...
4    non-fraud  organization maintains detailed record conting...


In [9]:

!pip install -U langchain-community

In [10]:
from langchain.docstore.document import Document

In [11]:
documents = []

# Iterate over rows using .rows() method
for i, row_tuple in df.iterrows():
    document = f"id:{i}\Fillings: {row_tuple[1]}\Fraud_Status: {row_tuple[0]}"
    documents.append(Document(page_content=document))

<ipython-input-11-e742069c45d6>:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  document = f"id:{i}\Fillings: {row_tuple[1]}\Fraud_Status: {row_tuple[0]}"


In [14]:
!pip install faiss-cpu

In [12]:
documents[0]

Document(metadata={}, page_content='id:0\\Fillings: company ensures working capital remains sufficient daily operations\\Fraud_Status: non-fraud')

In [13]:
from langchain_community.embeddings import HuggingFaceEmbeddings

<ipython-input-13-733749dc74cb>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hg_embeddings = HuggingFaceEmbeddings()
<ipython-input-13-733749dc74cb>:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  hg_embeddings = HuggingFaceEmbeddings()


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [16]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
from langchain.vectorstores import FAISS
# Build FAISS index
faiss_index = FAISS.from_documents(documents, embeddings)
faiss_index.save_local('docs/faiss_rag/')  # This will create index.faiss & index.pkl

In [18]:
from huggingface_hub import notebook_login
notebook_login(write_permission=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in 'notebook_login': write_permission. Will not be supported from version '1.0'.

Fine-grained tokens added complexity to the permissions, making it irrelevant to check if a token has 'write' access.
  warnings.warn(message, FutureWarning)


In [20]:
!pip install langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 4.8 MB/s eta 0:00:00


In [21]:
from google.colab import userdata
# LLM with function call
from langchain_groq import ChatGroq
import os
groq_api_key=userdata.get('groq_api_key')
os.environ["GROQ_API_KEY"]=groq_api_key
from langchain_groq import ChatGroq
llm = ChatGroq(
    groq_api_key=os.getenv("GROQ_API_KEY"),
    model_name='deepseek-r1-distill-llama-70b',
    temperature=0.1
)

llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7adbe4c3d110>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7adbe4b8b790>, model_name='deepseek-r1-distill-llama-70b', temperature=0.1, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [23]:
from IPython.display import display, Markdown
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [25]:
from langchain.schema import HumanMessage

question = "Please explain what EU AI Act is."
messages = [HumanMessage(content=question)]  # Wrap the question in a HumanMessage
response = llm(messages=messages)  # Pass messages to the llm

full_response =  f"Question: {question}\nAnswer: {response.content}"  # Access the response content
display(Markdown(colorize_text(full_response)))



**<font color='red'>Question:</font>** Please explain what EU AI Act is.


**<font color='green'>Answer:</font>** <think>
Okay, so I need to explain what the EU AI Act is. I remember hearing about it in the news, but I'm not exactly sure about the details. Let me try to piece this together.

First, I think it's some kind of regulation from the European Union related to artificial intelligence. The EU is known for having strict regulations, like GDPR for data protection, so maybe this is similar but for AI. I should probably start by confirming that it's an EU initiative.

I recall that the EU AI Act is proposed legislation, so it's not a law yet, but it's in the process of being passed. It's meant to regulate AI systems to ensure they're safe and respect human rights. That makes sense because AI can have significant impacts on society, both positive and negative.

I think the Act categorizes AI systems based on their potential risks. So, maybe there are different levels of regulation depending on how risky an AI application is. For example, high-risk areas might include things like law enforcement, healthcare, or transportation, where AI could have serious consequences if something goes wrong.

There's probably something about transparency and accountability. I remember reading that users should be informed when they're interacting with an AI, like in cases of deepfake videos or AI-generated content. That way, people know they're not dealing with a real person or authentic content.

Prohibitions are another aspect. The EU might be banning certain AI practices that are considered unethical or harmful. I think things like surveillance systems that violate privacy or AI that manipulates people in ways that undermine democracy could be prohibited.

I also remember something about conformity assessments. This might mean that before an AI system is used, especially in high-risk areas, it needs to go through some kind of evaluation to ensure it meets safety and ethical standards. This could involve third-party audits or certifications.

Enforcement is crucial for any regulation. The EU likely has measures in place to ensure compliance, such as fines or penalties for companies that don't adhere to the rules. National authorities would probably be responsible for overseeing this.

Public consultation and ongoing evaluation might be part of the process too. The EU would want to make sure that the regulations stay relevant as technology evolves, so there could be mechanisms for reviewing and updating the Act as needed.

I should also mention the timeline. I think the proposal was made in 2021, and it's still going through the legislative process. It might take a few years before it's fully implemented, but once it is, it could set a global standard for AI regulation, influencing other countries to follow suit.

Wait, I'm not entirely sure about the exact categories of risk. Maybe it's four levels: minimal risk, limited risk, high risk, and prohibited. Or perhaps it's just high-risk and prohibited. I should double-check that. Also, I'm a bit fuzzy on the specifics of the conformity assessments—what exactly do they entail? Are they mandatory for all AI systems or just high-risk ones?

Another point is the scope. Does the AI Act apply to all AI systems within the EU, or are there exceptions? For example, are certain sectors exempt, or are there size thresholds for companies? I think it's broader, covering any AI used in the EU, regardless of where the company is based.

I also wonder about the implications for businesses. How will this affect startups versus large corporations? Startups might find it harder to comply with stringent regulations, which could stifle innovation. On the other hand, clear guidelines might help build trust and ensure responsible AI development.

There's also the aspect of human oversight. The Act might require that AI systems have human oversight to prevent autonomous decisions that could harm individuals. This ties into accountability—ensuring that there's always a human responsible for AI actions.

I should also consider the ethical implications beyond just regulation. The EU is emphasizing ethical considerations in AI, which aligns with their values of protecting fundamental rights. This could influence how AI is developed not just in Europe but worldwide, as companies might adopt these standards to access the EU market.

In summary, the EU AI Act is a proposed regulation that aims to govern the development and deployment of AI systems within the EU. It focuses on risk-based approach, transparency, accountability, prohibitions on harmful AI, conformity assessments, and enforcement mechanisms. It's still in the legislative process, but once enacted, it could have a significant impact on the AI industry globally.
</think>

The EU AI Act is a proposed regulatory framework by the European Union aimed at governing the development and deployment of artificial intelligence (AI) systems. Here's a structured overview:

1. **Objective and Scope**: 
   - The Act seeks to ensure AI systems are safe, respect human rights, and adhere to ethical standards. It applies broadly to AI systems within the EU, regardless of the company's origin.

2. **Risk-Based Approach**:
   - AI systems are categorized based on their potential risks: minimal, limited, high, and prohibited. High-risk areas include sectors like healthcare, law enforcement, and transportation, where AI could have significant consequences.

3. **Transparency and Accountability**:
   - Users must be informed when interacting with AI, such as in cases of deepfakes or AI-generated content. This ensures awareness and transparency.

4. **Prohibitions**:
   - Certain harmful AI practices are banned, including surveillance that violates privacy and manipulative AI undermining democracy.

5. **Conformity Assessments**:
   - High-risk AI systems must undergo evaluations to meet safety and ethical standards, potentially involving third-party audits.

6. **Enforcement Mechanisms**:
   - Compliance is ensured through fines and penalties, overseen by national authorities.

7. **Public Consultation and Evaluation**:
   - The Act includes mechanisms for review and updates to keep pace with technological advancements.

8. **Timeline and Global Impact**:
   - Proposed in 2021, it is undergoing legislative processes. Once enacted, it could set a global standard for AI regulation.

9. **Implications for Businesses**:
   - While it may pose challenges for startups, it aims to build trust and ensure responsible AI development, influencing global practices.

10. **Ethical Considerations and Human Oversight**:
    - Emphasizes ethical AI development and requires human oversight to prevent harmful autonomous decisions.

In summary, the EU AI Act is a comprehensive approach to regulating AI, focusing on safety, ethics, and accountability, with potential global influence.

In [26]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFaceHub
from IPython.display import display, Markdown
import os
import warnings
warnings.filterwarnings('ignore')

In [27]:

# Define the prompt template
template = """
You are an Fraud Detection Expert in Financial Text Data, Analyse them and Predict is the Given Statement is Fraud or not?. If you don't know the answer, just say "Sorry, I Don't Know."
Question: {question}
Context: {context}
Answer:
"""
PROMPT = PromptTemplate(input_variables=["context", "query"], template=template)

# Ensure llm and langchain_chroma are properly initialized
retriever = faiss_index.as_retriever(search_kwargs={"k": 1})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=retriever, chain_type_kwargs={"prompt": PROMPT}
)

# Define your question
# question = "The company reported inflated revenues by including sales that never occurred."
question = "Financial records accurately reflect all expenses and liabilities."
# question = "Revenue was recognized prematurely before the actual sales occurred."
# question = "The balance sheet provides a true and fair view of the company’s financial position."

# Run the QA chain
try:
    result = qa_chain({"query": question})
    display(result)
except RuntimeError as e:
    print(f"RuntimeError encountered: {e}")

{'query': 'Financial records accurately reflect all expenses and liabilities.',
 'result': '<think>\nOkay, so I need to figure out whether the statement "Financial records accurately reflect all expenses and liabilities" is fraudulent or not based on the given context. Let me break this down step by step.\n\nFirst, the context provided is for id:124. The fillings mention that operational expenses are clearly categorized and recorded in the appropriate accounting period. The Fraud_Status is labeled as non-fraud. \n\nSo, the statement in question is about the accuracy of financial records concerning expenses and liabilities. The context says that the expenses are clearly categorized and recorded properly. That sounds like good accounting practices. When expenses are clearly categorized, it means they\'re organized in a way that makes sense, probably following standard accounting principles. Recording them in the appropriate period means that each expense is listed in the correct time fra

In [ ]:
f